In [ ]:
# first i have to connect with the GEE google earth engine by connecting to the account (getting the token ID)
#!pip install earthengine-api

In [ ]:
ee.Authenticate()

In [ ]:

ee.Initialize()

In [ ]:
import ee
import geopandas as gpd
#shapefile= gpd.read_file("Data/Data-Mohi-help/Data/Phnompenh_diss_UTM.shp")
shapefile=gpd.read_file("Raster to vector/water_clipped_from_2016.shp")
#shapefile_fire= gpd.read_file("EMSR704_AOI01_DEL_PRODUCT_v2/EMSR704_AOI01_DEL_PRODUCT_observedEventL_v1.shp")
print (shapefile)
shapefile.plot()

In [ ]:
#training the model 
import ee
import numpy as np
from sklearn.svm import SVC

# Initialize Earth Engine
#ee.Initialize()

# Load the training and validation tables
train_table = ee.FeatureCollection('projects/ee-alidia1998201811/assets/1000_training_points')
val_table = ee.FeatureCollection('projects/ee-alidia1998201811/assets/500_validationPoints')

# Extract training features and labels
train_features = np.array(train_table.aggregate_array('training17').getInfo())
for prop in ['training18', 'training19', 'training20', 'training21', 'training22','training23']:
    train_features = np.column_stack((train_features, np.array(train_table.aggregate_array(prop).getInfo())))
train_labels = np.array(train_table.aggregate_array('training16').getInfo())

# Train SVM classifier
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(train_features, train_labels)

print('Model trained successfully!')

In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import ee

# Initialize Earth Engine
ee.Initialize()

# Load the training and validation tables
train_table = ee.FeatureCollection('projects/ee-alidia1998201811/assets/1000_training_points')
val_table = ee.FeatureCollection('projects/ee-alidia1998201811/assets/500_validationPoints')

# Define training and validation properties per year
years = list(range(2016, 2024))  # From 2016 to 2023
train_properties = [f'training{year % 100}' for year in years]  # ['training16', 'training17', ..until 2023..]
valid_properties = [f'{year}_valid' for year in years]  # ['2016_valid', '2017_valid', ..until 2023..]

# Initialize lists to hold all training and validation data
all_train_features = []
all_train_labels = []
all_val_features = []
all_val_labels = []

# Loop through each year for independent training and validation
for train_prop, valid_prop, year in zip(train_properties, valid_properties, years):
    print(f"\nProcessing Year: {year}")

    # Extract training features and labels for the current year
    train_features = np.array(train_table.aggregate_array(train_prop).getInfo()).reshape(-1, 1).astype(float)
    train_labels = np.array(train_table.aggregate_array('training16').getInfo())  

    # Extract validation features and labels for the current year
    val_features = []
    val_labels = []

    for feature in val_table.getInfo()['features']:
        properties = feature['properties']
        if valid_prop in properties:  # Ensure the validation property exists, otherwise it will never work
            val_features.append([properties[valid_prop]])  
            val_labels.append(properties['2016_valid'])  

    val_features = np.array(val_features).astype(float)
    val_labels = np.array(val_labels)

    # Check if there's sufficient data
    if len(train_features) == 0 or len(val_features) == 0:
        print(f"Skipping {year} due to insufficient data.")
        continue

    # Append to the overall lists
    all_train_features.append(train_features)
    all_train_labels.append(train_labels)
    all_val_features.append(val_features)
    all_val_labels.append(val_labels)

# Combine all training and validation data
all_train_features = np.vstack(all_train_features)
all_train_labels = np.concatenate(all_train_labels)
all_val_features = np.vstack(all_val_features)
all_val_labels = np.concatenate(all_val_labels)

# Train the SVM model on the combined training data
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(all_train_features, all_train_labels)

# Predict validation labels on the combined validation data
val_predictions = svm_classifier.predict(all_val_features)

# Evaluate overall model accuracy
overall_accuracy = accuracy_score(all_val_labels, val_predictions)
print(f'Overall Validation Accuracy: {overall_accuracy:.4f}')

# Generate confusion matrix
cm = confusion_matrix(all_val_labels, val_predictions)

# Plot confusion matrix
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['No Water', 'Water'], yticklabels=['No Water', 'Water'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Overall Confusion Matrix')
plt.show()


## SVM & sentinel-2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ee
import geemap

# Initialize Earth Engine
ee.Initialize()

# Define Area of Interest (ROI)
roi = ee.FeatureCollection('projects/ee-alidia1998201811/assets/5kmFishnet').geometry()

# Cloud masking function for Sentinel-2




def maskS2clouds(image):
    qa = image.select('QA60')
    # Bits 10 and 11 are clouds and cirrus
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
        qa.bitwiseAnd(cirrusBitMask).eq(0)
    )
    return image.updateMask(mask).divide(10000)









# Load Sentinel-2 Data, apply cloud mask, and mosaic for full coverage
sentinel2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(roi) \
    .filterDate('2023-01-01', '2023-01-31') \
    .map(maskS2clouds) \
    #.select(['B3', 'B8'])
    .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12'])




band_names = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12']






# Mosaic to ensure full coverage over the ROI
sentinel2_mosaic = sentinel2.mosaic().clip(roi)

# Compute NDWI
ndwi = sentinel2_mosaic.normalizedDifference(['B3', 'B8']).rename('NDWI')

# Load Training Data
trainingData = ee.FeatureCollection('projects/ee-alidia1998201811/assets/1000_training_points')

# Function to aggregate water classes from training properties
def merge_classes(feature):
    water_label = ee.Number(0)
    for i in range(16, 24):  # Aggregating water classes
        water_label = water_label.add(feature.getNumber(f'training{i}'))
    return feature.set('water_class', ee.Algorithms.If(water_label.gt(0), 1, 0))

# Apply function to training data
trainingData = trainingData.map(merge_classes)

# Extract NDWI values for training
training = ndwi.sampleRegions(
    collection=trainingData,
    properties=['water_class'],
    scale=10
)

# Train SVM Classifier
svmClassifier = ee.Classifier.libsvm(kernelType='RBF', gamma=0.5, cost=10) \
    .train(features=training, classProperty='water_class', inputProperties=['band_names'])

# Apply SVM Model to classify Sentinel-2 data
classified = ndwi.classify(svmClassifier)

# Load Validation Data
validationData = ee.FeatureCollection('projects/ee-alidia1998201811/assets/500_validationPoints')

# Sample the classified image using the validation data
years = [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]  # Years of interest
validation_samples = classified.sampleRegions(
    collection=validationData,
    properties=[f"{year}_valid" for year in years],
    scale=10
)

# Check extracted validation samples
validation_samples_info = validation_samples.getInfo()
print("Sample of validation samples:", validation_samples_info)

# Select the correct validation column (e.g., latest year 2023)
confusionMatrix = validation_samples.errorMatrix(
    actual='2023_valid',  # Adjust based on the chosen validation year
    predicted='classification'
)

# Extract confusion matrix values as a 2D array
confusion_values = np.array(confusionMatrix.getInfo())

# Check if confusion matrix is correctly structured
print("Confusion matrix values:", confusion_values)
if confusion_values.shape != (2, 2):
    print("Warning: Confusion matrix is not 2x2, check your classes!")

# Define labels
labels = ['Non-water (0)', 'Water (1)']

# Plot Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_values, annot=True, fmt='d', cmap='Blues',
            xticklabels=labels, yticklabels=labels)
plt.title('Sentinel-2 Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


In [ ]:
# Example: choose a specific month (e.g., March 2023)
start_date = '2019-01-01'
end_date   = '2019-01-31'

# Filter Sentinel-2 collection for that month
monthly_collection = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(roi) \
    .filterDate(start_date, end_date) \
    .map(maskS2clouds) \
    .select(['B3', 'B8', 'B4', 'B2'])

# Mosaic all images for full coverage
specific_image = monthly_collection.mosaic().clip(roi)

# -------------------------------
# STEP 2: Compute NDWI for the mosaic
# -------------------------------
ndwi_single = specific_image.normalizedDifference(['B3', 'B8']).rename('NDWI').clip(roi)

# -------------------------------
# STEP 3: Apply classifier
# -------------------------------
classified_single = ndwi_single.classify(svmClassifier)

# -------------------------------
# STEP 4: Visualize
# -------------------------------
Map = geemap.Map()
Map.centerObject(roi, 10)

Map.addLayer(specific_image, 
             {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}, 
             'RGB (Monthly Mosaic)')
Map.addLayer(ndwi_single, 
             {'min': -1, 'max': 1, 'palette': ['brown', 'blue']}, 
             'NDWI (Monthly)')
Map.addLayer(classified_single, 
             {'min': 0, 'max': 1, 'palette': ['gray', 'blue']}, 
             'Classified (Monthly)')

Map.addLayerControl()
Map


## SVM & Sentinel-1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ee
import geemap

# Initialize Earth Engine
ee.Initialize()

# Define Area of Interest (ROI)
roi = ee.FeatureCollection('projects/ee-alidia1998201811/assets/5kmFishnet').geometry()

# Load Sentinel-1 Data
sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(roi) \
    .filterDate('2023-01-01', '2023-12-31') \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
    .filter(ee.Filter.eq('resolution_meters', 10)) \
    .select(['VV', 'VH'])

# Compute median composite
sentinel1_median = sentinel1.median().clip(roi)

# Load Training Data
trainingData = ee.FeatureCollection('projects/ee-alidia1998201811/assets/1000_training_points')

# Function to safely get properties
def get_property_or_default(feature, property_name):
    return ee.Algorithms.If(feature.get(property_name), feature.get(property_name), 0)

# Function to merge water training columns
def merge_classes(feature):
    water_label = ee.Number(0)
    #  water class is determined from columns 2016 to 2023
    for i in range(16, 24): 
        water_label = water_label.add(get_property_or_default(feature, f'training{i}'))
    return feature.set('water_class', ee.Algorithms.If(water_label.gt(0), 1, 0))

# Apply function to training data
trainingData = trainingData.map(merge_classes)

# Extract Sentinel-1 values for training
training = sentinel1_median.sampleRegions(
    collection=trainingData,
    properties=['water_class'],
    scale=10
)

# Train SVM Classifier
svmClassifier = ee.Classifier.libsvm(
    kernelType='RBF',
    gamma=0.5,
    cost=10
).train(features=training, classProperty='water_class', inputProperties=['VV', 'VH'])

# Apply SVM Model
classified = sentinel1_median.classify(svmClassifier)

# Load Validation Data
validationData = ee.FeatureCollection('projects/ee-alidia1998201811/assets/500_validationPoints')

# Sample the classified image using the validation data
# Specify the years of interest for validation
years = [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
validation_samples = classified.sampleRegions(
    collection=validationData,
    properties=[f"{year}_valid" for year in years],
    scale=10
)

# Check the structure and class distribution in validation samples
validation_samples_info = validation_samples.getInfo()
print("Sample of validation samples:", validation_samples_info)

# Assuming we want to classify based on one of the columns, say 2023_valid
# we can adjust this based on the actual label column you want to compare against.
# Let's assume we are comparing against the last valid year column, `2023_valid`.
confusionMatrix = validation_samples.errorMatrix(
    actual='2023_valid',  # Change as necessary if you're using a different year
    predicted='classification'
)

# Extract the confusion matrix values as a 2D array
confusion_values = np.array(confusionMatrix.getInfo())

# Check the structure of confusion_values
print("Confusion matrix values:", confusion_values)

# Ensure confusion values are properly shaped (should be 2x2)
if confusion_values.shape != (2, 2):
    print("Warning: Confusion matrix is not 2x2, check your classes!")

# Define labels for the classes
labels = ['Non-water (0)', 'Water (1)']  # Adjust based on your classes

# Plotting the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_values, annot=True, fmt='d', cmap='Blues',
            xticklabels=labels, yticklabels=labels)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


In [ ]:

start_date = '2016-01-01'
end_date = '2016-01-30'

sentinel1_month = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(roi) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
    .filter(ee.Filter.eq('resolution_meters', 10)) \
    .select(['VV', 'VH']) \
    .median() \
    .clip(roi)

classified_month = sentinel1_month.classify(svmClassifier)

Map = geemap.Map()
Map.addLayer(sentinel1_month.select('VV'), {'min': -25, 'max': 5}, 'Sentinel-1 VV (monthly)')
Map.addLayer(classified_month, vis_params_class, 'Classification (monthly)')
Map.centerObject(roi, 9)
Map





